In [20]:
import math
import copy

In [21]:
class Vector:
    def __init__(self, data):
        if isinstance(data, int):
            self.size = data
            self.data = [0.0] * data
        else:
            self.size = len(data)
            self.data = [float(x) for x in data]

    def __getitem__(self, index):
        return self.data[index]

    def __setitem__(self, index, value):
        self.data[index] = value

    def __str__(self):
        elements = ", ".join(f"{x:.4f}" for x in self.data)
        return f"[{elements}]"

    def norm1(self):
        total = 0.0
        for x in self.data:
            total += abs(x)
        return total

    def norm2(self):
        total = 0.0
        for x in self.data:
            total += x * x
        return math.sqrt(total)

    def norm_inf(self):
        max_val = 0.0
        for x in self.data:
            if abs(x) > max_val:
                max_val = abs(x)
        return max_val

In [22]:
class Matrix:
    def __init__(self, data):
        if isinstance(data, tuple):
            self.rows, self.cols = data
            self.data = [[0.0] * self.cols for _ in range(self.rows)]
        else:
            self.rows = len(data)
            self.cols = len(data[0])
            self.data = data

    def __getitem__(self, indices):
        i, j = indices
        return self.data[i][j]

    def __setitem__(self, indices, value):
        i, j = indices
        self.data[i][j] = value

    def __str__(self):
        lines = []
        for row in self.data:
            elements = ", ".join(f"{x:8.4f}" for x in row)
            lines.append(f"[{elements}]")
        return "\n".join(lines)

    def norm1(self):
        max_sum = 0.0
        for j in range(self.cols):
            col_sum = 0.0
            for i in range(self.rows):
                col_sum += abs(self[i, j])
            if col_sum > max_sum:
                max_sum = col_sum
        return max_sum

    def norm_inf(self):
        max_sum = 0.0
        for i in range(self.rows):
            row_sum = 0.0
            for j in range(self.cols):
                row_sum += abs(self[i, j])
            if row_sum > max_sum:
                max_sum = row_sum
        return max_sum

    def norm_frobenius(self):
        total = 0.0
        for i in range(self.rows):
            for j in range(self.cols):
                total += self[i, j] * self[i, j]
        return math.sqrt(total)


    def transpose(self):
        result = Matrix((self.cols, self.rows))
        for i in range(self.rows):
            for j in range(self.cols):
                result[j, i] = self[i, j]
        return result

    def __mul__(self, other):
        if isinstance(other, Vector):
            if self.cols != other.size:
                raise ValueError('Несовместимые размеры')

            result = Vector(self.rows)
            for i in range(self.rows):
                total = 0.0
                for j in range(self.cols):
                    total += self[i, j] * other[j]
                result[i] = total
            return result

    def inverse(self):
        if self.rows != self.cols:
            raise ValueError("Матрица должна быть квадратной для обращения")

        n = self.rows
        augmented = Matrix((n, 2 * n))

        for i in range(n):
            for j in range(n):
                augmented[i, j] = self[i, j]

        for i in range(n):
            augmented[i, n + i] = 1.0

        for col in range(n):
            max_row = col
            for row in range(col + 1, n):
                if abs(augmented[row, col]) > abs(augmented[max_row, col]):
                    max_row = row

            if max_row != col:
                for j in range(2 * n):
                    augmented[col, j], augmented[max_row, j] = augmented[max_row, j], augmented[col, j]

            pivot = augmented[col, col]
            if abs(pivot) < 1e-10:
                raise ValueError("Матрица вырождена, обратной не существует")

            for j in range(2 * n):
                augmented[col, j] /= pivot

            for i in range(n):
                if i != col:
                    factor = augmented[i, col]
                    for j in range(2 * n):
                        augmented[i, j] -= factor * augmented[col, j]

        inverse_mat = Matrix((n, n))
        for i in range(n):
            for j in range(n):
                inverse_mat[i, j] = augmented[i, n + j]

        return inverse_mat

    def determinant(self):
        if self.rows != self.cols:
            raise ValueError("Матрица должна быть квадратной")

        n = self.rows
        mat_copy = copy.deepcopy(self.data)
        det = 1.0

        for col in range(n):
            max_row = col
            for row in range(col + 1, n):
                if abs(mat_copy[row][col]) > abs(mat_copy[max_row][col]):
                    max_row = row

            if max_row != col:
                mat_copy[col], mat_copy[max_row] = mat_copy[max_row], mat_copy[col]
                det *= -1

            pivot = mat_copy[col][col]
            if abs(pivot) < 1e-10:
                return 0.0

            det *= pivot

            for i in range(col + 1, n):
                factor = mat_copy[i][col] / pivot
                for j in range(col + 1, n):
                    mat_copy[i][j] -= factor * mat_copy[col][j]

        return det

    def condition_number(self, norm_type='2'):
        try:
            A_inv = self.inverse()
        except ValueError:
            return float('inf')

        if norm_type == '1':
            return self.norm1() * A_inv.norm1()
        elif norm_type == '2':
            return self.norm_frobenius() * A_inv.norm_frobenius()
        elif norm_type == 'inf':
            return self.norm_inf() * A_inv.norm_inf()
        elif norm_type == 'fro':
            return self.norm_frobenius() * A_inv.norm_frobenius()
        else:
            raise ValueError("Неизвестный тип нормы")

    def is_symmetric(self):
        if self.rows != self.cols:
            return False

        for i in range(self.rows):
            for j in range(i + 1, self.cols):
                if abs(self[i, j] - self[j, i]) > 1e-10:
                    return False
        return True

    def eigenvals_approximation(self):
        if not self.is_symmetric():
            print("Внимание: матрица не симметрична, результаты могут быть неточными")

        trace = sum(self[i, i] for i in range(self.rows))
        fro_norm = self.norm_frobenius()

        max_eigen_approx = fro_norm
        min_eigen_approx = trace / self.rows

        return {
            'max_approx': max_eigen_approx,
            'min_approx': min_eigen_approx,
            'trace': trace
        }

In [23]:
print("=== ТЕСТИРУЕМ ВЕКТОР ===")
v = Vector([1, -2, 3, -4])
print(f"Вектор v = {v}")
print(f"Норма L1: {v.norm1()}")
print(f"Норма L2: {v.norm2():.4f}")
print(f"Норма L∞: {v.norm_inf()}")

print("\n=== ТЕСТИРУЕМ МАТРИЦУ ===")
A = Matrix([[1, 2, 3], [4, 5, 6]])
print("Матрица A:")
print(A)
print(f"Норма L1: {A.norm1()}")
print(f"Норма L∞: {A.norm_inf()}")
print(f"Норма Фробениуса: {A.norm_frobenius():.4f}")

print("\n=== ТЕСТИРУЕМ ОПЕРАЦИИ ===")
print("Транспонированная A:")
A_T = A.transpose()
print(A_T)

print("\nУмножение матрицы на вектор:")
v2 = Vector([1, 0, 2])
result = A * v2
print(f"A * v = {result}")


print("\n" + "="*50)
print("ТЕСТИРУЕМ ОБРАЩЕНИЕ МАТРИЦ И ОБУСЛОВЛЕННОСТЬ")
print("="*50)

# Тест 1: Хорошо обусловленная матрица
print("\n1. ХОРОШО ОБУСЛОВЛЕННАЯ МАТРИЦА:")
A_good = Matrix([[4, 1], [1, 3]])
print("Матрица A:")
print(A_good)

try:
    A_inv = A_good.inverse()
    print("Обратная матрица A⁻¹:")
    print(A_inv)

    identity_check = A_good * A_inv
    print("Проверка A * A⁻¹ (должна быть единичной):")
    print(identity_check)

    cond = A_good.condition_number('1')
    print(f"Число обусловленности (L1): {cond:.4f}")

except ValueError as e:
    print(f"Ошибка: {e}")

print("\n2. ПЛОХО ОБУСЛОВЛЕННАЯ МАТРИЦА:")
A_bad = Matrix([[1, 1], [1, 1.0001]])
print("Матрица B:")
print(A_bad)

try:
    B_inv = A_bad.inverse()
    print("Обратная матрица B⁻¹:")
    print(B_inv)

    cond = A_bad.condition_number('1')
    print(f"Число обусловленности (L1): {cond:.4f}")

except ValueError as e:
    print(f"Ошибка: {e}")

print("\n3. ОПРЕДЕЛИТЕЛИ:")
for name, mat in [("A_good", A_good), ("A_bad", A_bad)]:
    det = mat.determinant()
    print(f"det({name}) = {det:.6f}")

print("\n4. СИММЕТРИЧНОСТЬ И СОБСТВЕННЫЕ ЗНАЧЕНИЯ:")
print(f"A_good симметрична: {A_good.is_symmetric()}")
print(f"A_bad симметрична: {A_bad.is_symmetric()}")

eigen_info = A_good.eigenvals_approximation()
print("Приближенные собственные значения A_good:")
print(f"  Максимальное: {eigen_info['max_approx']:.4f}")
print(f"  Минимальное: {eigen_info['min_approx']:.4f}")
print(f"  След: {eigen_info['trace']:.4f}")

print("\n5. СРАВНЕНИЕ ЧИСЕЛ ОБУСЛОВЛЕННОСТИ:")
matrices = [("Хорошая", A_good), ("Плохая", A_bad)]
norms = ['1', 'inf', 'fro']

for name, mat in matrices:
    print(f"\n{name} матрица:")
    for norm in norms:
        try:
            cond = mat.condition_number(norm)
            print(f"  μ_{norm} = {cond:.2f}")
        except:
            print(f"  μ_{norm} = не вычислено")

=== ТЕСТИРУЕМ ВЕКТОР ===
Вектор v = [1.0000, -2.0000, 3.0000, -4.0000]
Норма L1: 10.0
Норма L2: 5.4772
Норма L∞: 4.0

=== ТЕСТИРУЕМ МАТРИЦУ ===
Матрица A:
[  1.0000,   2.0000,   3.0000]
[  4.0000,   5.0000,   6.0000]
Норма L1: 9.0
Норма L∞: 15.0
Норма Фробениуса: 9.5394

=== ТЕСТИРУЕМ ОПЕРАЦИИ ===
Транспонированная A:
[  1.0000,   4.0000]
[  2.0000,   5.0000]
[  3.0000,   6.0000]

Умножение матрицы на вектор:
A * v = [7.0000, 16.0000]

ТЕСТИРУЕМ ОБРАЩЕНИЕ МАТРИЦ И ОБУСЛОВЛЕННОСТЬ

1. ХОРОШО ОБУСЛОВЛЕННАЯ МАТРИЦА:
Матрица A:
[  4.0000,   1.0000]
[  1.0000,   3.0000]
Обратная матрица A⁻¹:
[  0.2727,  -0.0909]
[ -0.0909,   0.3636]
Проверка A * A⁻¹ (должна быть единичной):
None
Число обусловленности (L1): 2.2727

2. ПЛОХО ОБУСЛОВЛЕННАЯ МАТРИЦА:
Матрица B:
[  1.0000,   1.0000]
[  1.0000,   1.0001]
Обратная матрица B⁻¹:
[10001.0000, -10000.0000]
[-10000.0000, 10000.0000]
Число обусловленности (L1): 40004.0001

3. ОПРЕДЕЛИТЕЛИ:
det(A_good) = 11.000000
det(A_bad) = 0.000100

4. СИММЕТРИЧНОСТЬ 